<a href="https://colab.research.google.com/github/samantha8990/finalproject/blob/main/Solved_Unsolved_MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

!pip install pyspark

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
     |████████████████████████████████| 212.4 MB 65 kB/s 
     |████████████████████████████████| 198 kB 65.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=c34a10b412168d251c453ce5985acb5f4af6689ddcdb4fb78dfd4273452f7362
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkDates").getOrCreate()

In [3]:
# Load in data
from pyspark import SparkFiles
url ="https://murderanalytics.s3.us-west-1.amazonaws.com/CleanData.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("CleanData.csv"), sep=",", header=True, inferSchema=True)
df.show()

+---+-------------+------+----+--------+--------------------+------+------+------+-------+--------------------+--------------------+--------------------+--------------------+
|_c0|     CNTYFIPS|Solved|Year|   Month|            Homicide|VicAge|VicSex|OffAge| OffSex|              Weapon|        Relationship|        Circumstance|           Situation|
+---+-------------+------+----+--------+--------------------+------+------+------+-------+--------------------+--------------------+--------------------+--------------------+
|  0|Anchorage, AK|   Yes|2016| January|Murder and non-ne...|    40|Female|    15|   Male|Firearm, type not...|              Sister|Circumstances und...|Multiple victims/...|
|  1|Anchorage, AK|   Yes|2016| January|Murder and non-ne...|    18|Female|    15|   Male|Firearm, type not...|              Sister|Circumstances und...|Multiple victims/...|
|  2|Anchorage, AK|   Yes|2016| January|Murder and non-ne...|    49|  Male|    34|   Male|Firearm, type not...|            St

Data Pre-Processing

In [4]:
import pandas as pd
data = df.toPandas()
data.head()

,_c0,CNTYFIPS,Solved,Year,Month,Homicide,VicAge,VicSex,OffAge,OffSex,Weapon,Relationship,Circumstance,Situation
0,0,"Anchorage, AK",Yes,2016,January,Murder and non-negligent manslaughter,40,Female,15,Male,"Firearm, type not stated",Sister,Circumstances undetermined,Multiple victims/single offender
1,1,"Anchorage, AK",Yes,2016,January,Murder and non-negligent manslaughter,18,Female,15,Male,"Firearm, type not stated",Sister,Circumstances undetermined,Multiple victims/single offender
2,2,"Anchorage, AK",Yes,2016,January,Murder and non-negligent manslaughter,49,Male,34,Male,"Firearm, type not stated",Stranger,Felon killed by police,Single victim/multiple offenders
3,3,"Anchorage, AK",Yes,2016,January,Murder and non-negligent manslaughter,32,Female,33,Male,Other or type unknown,Wife,Circumstances undetermined,Single victim/single offender
4,4,"Anchorage, AK",No,2016,January,Murder and non-negligent manslaughter,20,Male,999,Unknown,"Firearm, type not stated",Relationship not determined,Circumstances undetermined,Multiple victims/unknown offender(s)


In [5]:
MLData = data[["CNTYFIPS","Solved", "Month", "VicAge", "OffSex", "Weapon", "Relationship"]]
MLData.head()

,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife
4,"Anchorage, AK",No,January,20,Unknown,"Firearm, type not stated",Relationship not determined


Using Machine Learning we will learn how you are going to DIE.



In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow
tensorflow.keras.__version__
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Create numerical encoding for each unique column

In [7]:
#Solved
solved = np.unique(MLData['Solved'])
solved

array(['No', 'Yes'], dtype=object)

In [8]:
sle = LabelEncoder()
solved_labels = sle.fit_transform(MLData['Solved'])
solved_mappings = {index: label for index, label in 
                  enumerate(sle.classes_)}
solved_mappings

{0: 'No', 1: 'Yes'}

In [9]:
MLData['SolvedLabel'] = solved_labels
MLData.head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship,SolvedLabel
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister,1
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister,1
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger,1
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife,1
4,"Anchorage, AK",No,January,20,Unknown,"Firearm, type not stated",Relationship not determined,0
5,"Anchorage, AK",No,January,19,Unknown,"Firearm, type not stated",Relationship not determined,0
6,"Anchorage, AK",Yes,January,2,Female,"Handgun - pistol, revolver, etc",Son,1
7,"Anchorage, AK",Yes,February,30,Male,"Firearm, type not stated",Stranger,1
8,"Anchorage, AK",No,March,24,Unknown,"Firearm, type not stated",Relationship not determined,0
9,"Anchorage, AK",Yes,March,30,Male,"Firearm, type not stated",Friend,1


In [10]:
#CNTYFIPS encoding
location = np.unique(MLData['CNTYFIPS'])
location


array(['Abbeville, SC', 'Acadia, LA', 'Accomack, VA', ..., 'Young, TX',
       'Yuba, CA', 'Yuma, AZ'], dtype=object)

In [11]:
cle = LabelEncoder()
location_labels = cle.fit_transform(MLData['CNTYFIPS'])
location_mappings = {index: label for index, label in 
                  enumerate(cle.classes_)}
location_mappings

{0: 'Abbeville, SC',
 1: 'Acadia, LA',
 2: 'Accomack, VA',
 3: 'Ada, ID',
 4: 'Adair, KY',
 5: 'Adair, MO',
 6: 'Adair, OK',
 7: 'Adams, CO',
 8: 'Adams, IL',
 9: 'Adams, MS',
 10: 'Adams, NE',
 11: 'Adams, OH',
 12: 'Adams, PA',
 13: 'Adams, WA',
 14: 'Adams, WI',
 15: 'Addison, VT',
 16: 'Aiken, SC',
 17: 'Aitkin, MN',
 18: 'Alachua, FL',
 19: 'Alamance, NC',
 20: 'Alameda, CA',
 21: 'Alamosa, CO',
 22: 'Albany, NY',
 23: 'Albany, WY',
 24: 'Albemarle, VA',
 25: 'Alexander, NC',
 26: 'Alexandria, VA',
 27: 'Alfalfa, OK',
 28: 'Alger, MI',
 29: 'Allegan, MI',
 30: 'Allegany, MD',
 31: 'Allegany, NY',
 32: 'Allegheny, PA',
 33: 'Allen, IN',
 34: 'Allen, KS',
 35: 'Allen, KY',
 36: 'Allen, LA',
 37: 'Allen, OH',
 38: 'Allendale, SC',
 39: 'Alpena, MI',
 40: 'Amador, CA',
 41: 'Amelia, VA',
 42: 'Amherst, VA',
 43: 'Anchorage, AK',
 44: 'Anderson, KS',
 45: 'Anderson, SC',
 46: 'Anderson, TN',
 47: 'Anderson, TX',
 48: 'Andrew, MO',
 49: 'Andrews, TX',
 50: 'Androscoggin, ME',
 51: 'Ange

In [12]:
MLData['LocationLabel'] = location_labels
MLData.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship,SolvedLabel,LocationLabel
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister,1,43
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister,1,43
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger,1,43
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife,1,43
4,"Anchorage, AK",No,January,20,Unknown,"Firearm, type not stated",Relationship not determined,0,43
5,"Anchorage, AK",No,January,19,Unknown,"Firearm, type not stated",Relationship not determined,0,43
6,"Anchorage, AK",Yes,January,2,Female,"Handgun - pistol, revolver, etc",Son,1,43
7,"Anchorage, AK",Yes,February,30,Male,"Firearm, type not stated",Stranger,1,43
8,"Anchorage, AK",No,March,24,Unknown,"Firearm, type not stated",Relationship not determined,0,43
9,"Anchorage, AK",Yes,March,30,Male,"Firearm, type not stated",Friend,1,43


In [13]:
#Month encoding
Month = np.unique(MLData['Month'])
Month


array(['April', 'August', 'December', 'February', 'January', 'July',
       'June', 'March', 'May', 'November', 'October', 'September'],
      dtype=object)

In [14]:
mle = LabelEncoder()
month_labels = mle.fit_transform(MLData['Month'])
month_mappings = {index: label for index, label in 
                  enumerate(mle.classes_)}
month_mappings

{0: 'April',
 1: 'August',
 2: 'December',
 3: 'February',
 4: 'January',
 5: 'July',
 6: 'June',
 7: 'March',
 8: 'May',
 9: 'November',
 10: 'October',
 11: 'September'}

In [15]:
MLData['MonthLabel'] = month_labels
MLData.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship,SolvedLabel,LocationLabel,MonthLabel
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister,1,43,4
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister,1,43,4
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger,1,43,4
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife,1,43,4
4,"Anchorage, AK",No,January,20,Unknown,"Firearm, type not stated",Relationship not determined,0,43,4
5,"Anchorage, AK",No,January,19,Unknown,"Firearm, type not stated",Relationship not determined,0,43,4
6,"Anchorage, AK",Yes,January,2,Female,"Handgun - pistol, revolver, etc",Son,1,43,4
7,"Anchorage, AK",Yes,February,30,Male,"Firearm, type not stated",Stranger,1,43,3
8,"Anchorage, AK",No,March,24,Unknown,"Firearm, type not stated",Relationship not determined,0,43,7
9,"Anchorage, AK",Yes,March,30,Male,"Firearm, type not stated",Friend,1,43,7


skipping VicAge due to it already being a numerical value

In [16]:
#OffSex encoding
Gender = np.unique(MLData['OffSex'])
Gender

array(['Female', 'Male', 'Unknown'], dtype=object)

In [17]:
gle = LabelEncoder()
gender_labels = gle.fit_transform(MLData['OffSex'])
gender_mappings = {index: label for index, label in 
                  enumerate(gle.classes_)}
gender_mappings

{0: 'Female', 1: 'Male', 2: 'Unknown'}

In [18]:
MLData['GenderLabel'] = gender_labels
MLData.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship,SolvedLabel,LocationLabel,MonthLabel,GenderLabel
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister,1,43,4,1
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister,1,43,4,1
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger,1,43,4,1
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife,1,43,4,1
4,"Anchorage, AK",No,January,20,Unknown,"Firearm, type not stated",Relationship not determined,0,43,4,2
5,"Anchorage, AK",No,January,19,Unknown,"Firearm, type not stated",Relationship not determined,0,43,4,2
6,"Anchorage, AK",Yes,January,2,Female,"Handgun - pistol, revolver, etc",Son,1,43,4,0
7,"Anchorage, AK",Yes,February,30,Male,"Firearm, type not stated",Stranger,1,43,3,1
8,"Anchorage, AK",No,March,24,Unknown,"Firearm, type not stated",Relationship not determined,0,43,7,2
9,"Anchorage, AK",Yes,March,30,Male,"Firearm, type not stated",Friend,1,43,7,1


In [19]:
#Weapon encoding
Weapon = np.unique(MLData['Weapon'])
Weapon

array(['Asphyxiation - includes death by gas',
       'Blunt object - hammer, club, etc', 'Drowning', 'Explosives',
       'Fire', 'Firearm, type not stated',
       'Handgun - pistol, revolver, etc', 'Knife or cutting instrument',
       'Narcotics or drugs, sleeping pills', 'Other gun',
       'Other or type unknown', 'Personal weapons, includes beating',
       'Poison - does not include gas', 'Pushed or thrown out window',
       'Rifle', 'Shotgun', 'Strangulation - hanging'], dtype=object)

In [20]:
wle = LabelEncoder()
weapon_labels = wle.fit_transform(MLData['Weapon'])
weapon_mappings = {index: label for index, label in 
                  enumerate(wle.classes_)}
weapon_mappings

{0: 'Asphyxiation - includes death by gas',
 1: 'Blunt object - hammer, club, etc',
 2: 'Drowning',
 3: 'Explosives',
 4: 'Fire',
 5: 'Firearm, type not stated',
 6: 'Handgun - pistol, revolver, etc',
 7: 'Knife or cutting instrument',
 8: 'Narcotics or drugs, sleeping pills',
 9: 'Other gun',
 10: 'Other or type unknown',
 11: 'Personal weapons, includes beating',
 12: 'Poison - does not include gas',
 13: 'Pushed or thrown out window',
 14: 'Rifle',
 15: 'Shotgun',
 16: 'Strangulation - hanging'}

In [21]:
MLData['WeaponLabel'] = weapon_labels
MLData.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship,SolvedLabel,LocationLabel,MonthLabel,GenderLabel,WeaponLabel
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister,1,43,4,1,5
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister,1,43,4,1,5
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger,1,43,4,1,5
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife,1,43,4,1,10
4,"Anchorage, AK",No,January,20,Unknown,"Firearm, type not stated",Relationship not determined,0,43,4,2,5
5,"Anchorage, AK",No,January,19,Unknown,"Firearm, type not stated",Relationship not determined,0,43,4,2,5
6,"Anchorage, AK",Yes,January,2,Female,"Handgun - pistol, revolver, etc",Son,1,43,4,0,6
7,"Anchorage, AK",Yes,February,30,Male,"Firearm, type not stated",Stranger,1,43,3,1,5
8,"Anchorage, AK",No,March,24,Unknown,"Firearm, type not stated",Relationship not determined,0,43,7,2,5
9,"Anchorage, AK",Yes,March,30,Male,"Firearm, type not stated",Friend,1,43,7,1,5


In [22]:
#Relationship encoding
Relationship = np.unique(MLData['Relationship'])
Relationship

array(['Acquaintance', 'Boyfriend', 'Brother', 'Common-law husband',
       'Common-law wife', 'Daughter', 'Employee', 'Employer',
       'Ex-husband', 'Ex-wife', 'Father', 'Friend', 'Girlfriend',
       'Homosexual relationship', 'Husband', 'In-law', 'Mother',
       'Neighbor', 'Other - known to victim', 'Other family',
       'Relationship not determined', 'Sister', 'Son', 'Stepdaughter',
       'Stepfather', 'Stepmother', 'Stepson', 'Stranger', 'Wife'],
      dtype=object)

In [23]:
rle = LabelEncoder()
relationship_labels = rle.fit_transform(MLData['Relationship'])
relationship_mappings = {index: label for index, label in 
                  enumerate(rle.classes_)}
relationship_mappings

{0: 'Acquaintance',
 1: 'Boyfriend',
 2: 'Brother',
 3: 'Common-law husband',
 4: 'Common-law wife',
 5: 'Daughter',
 6: 'Employee',
 7: 'Employer',
 8: 'Ex-husband',
 9: 'Ex-wife',
 10: 'Father',
 11: 'Friend',
 12: 'Girlfriend',
 13: 'Homosexual relationship',
 14: 'Husband',
 15: 'In-law',
 16: 'Mother',
 17: 'Neighbor',
 18: 'Other - known to victim',
 19: 'Other family',
 20: 'Relationship not determined',
 21: 'Sister',
 22: 'Son',
 23: 'Stepdaughter',
 24: 'Stepfather',
 25: 'Stepmother',
 26: 'Stepson',
 27: 'Stranger',
 28: 'Wife'}

In [24]:
MLData['RelationshipLabel'] = relationship_labels
MLData.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CNTYFIPS,Solved,Month,VicAge,OffSex,Weapon,Relationship,SolvedLabel,LocationLabel,MonthLabel,GenderLabel,WeaponLabel,RelationshipLabel
0,"Anchorage, AK",Yes,January,40,Male,"Firearm, type not stated",Sister,1,43,4,1,5,21
1,"Anchorage, AK",Yes,January,18,Male,"Firearm, type not stated",Sister,1,43,4,1,5,21
2,"Anchorage, AK",Yes,January,49,Male,"Firearm, type not stated",Stranger,1,43,4,1,5,27
3,"Anchorage, AK",Yes,January,32,Male,Other or type unknown,Wife,1,43,4,1,10,28
4,"Anchorage, AK",No,January,20,Unknown,"Firearm, type not stated",Relationship not determined,0,43,4,2,5,20
5,"Anchorage, AK",No,January,19,Unknown,"Firearm, type not stated",Relationship not determined,0,43,4,2,5,20
6,"Anchorage, AK",Yes,January,2,Female,"Handgun - pistol, revolver, etc",Son,1,43,4,0,6,22
7,"Anchorage, AK",Yes,February,30,Male,"Firearm, type not stated",Stranger,1,43,3,1,5,27
8,"Anchorage, AK",No,March,24,Unknown,"Firearm, type not stated",Relationship not determined,0,43,7,2,5,20
9,"Anchorage, AK",Yes,March,30,Male,"Firearm, type not stated",Friend,1,43,7,1,5,11


In [25]:
#Split Dataframe to encoded/non encoded
encoded_DF = MLData[["SolvedLabel", "LocationLabel", "MonthLabel", "GenderLabel", "WeaponLabel", "RelationshipLabel"]]
encoded_DF.head()

,SolvedLabel,LocationLabel,MonthLabel,GenderLabel,WeaponLabel,RelationshipLabel
0,1,43,4,1,5,21
1,1,43,4,1,5,21
2,1,43,4,1,5,27
3,1,43,4,1,10,28
4,0,43,4,2,5,20


Will the murder be solved or unsolved?

In [26]:
X = encoded_DF.drop("SolvedLabel", axis=1)
y = encoded_DF["SolvedLabel"]
print(X.shape, y.shape)

(69623, 5) (69623,)


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [28]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [30]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [35]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=5))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [36]:
 # Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               600       
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 10,902
Trainable params: 10,902
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
1632/1632 - 3s - loss: 0.0349 - accuracy: 0.9910
Epoch 2/60
1632/1632 - 2s - loss: 0.0143 - accuracy: 0.9977
Epoch 3/60
1632/1632 - 2s - loss: 0.0142 - accuracy: 0.9978
Epoch 4/60
1632/1632 - 2s - loss: 0.0140 - accuracy: 0.9978
Epoch 5/60
1632/1632 - 2s - loss: 0.0139 - accuracy: 0.9979
Epoch 6/60
1632/1632 - 2s - loss: 0.0137 - accuracy: 0.9979
Epoch 7/60
1632/1632 - 2s - loss: 0.0135 - accuracy: 0.9979
Epoch 8/60
1632/1632 - 2s - loss: 0.0136 - accuracy: 0.9978
Epoch 9/60
1632/1632 - 2s - loss: 0.0133 - accuracy: 0.9979
Epoch 10/60
1632/1632 - 2s - loss: 0.0131 - accuracy: 0.9978
Epoch 11/60
1632/1632 - 2s - loss: 0.0132 - accuracy: 0.9979
Epoch 12/60
1632/1632 - 2s - loss: 0.0132 - accuracy: 0.9979
Epoch 13/60
1632/1632 - 2s - loss: 0.0131 - accuracy: 0.9979
Epoch 14/60
1632/1632 - 2s - loss: 0.0131 - accuracy: 0.9979
Epoch 15/60
1632/1632 - 2s - loss: 0.0128 - accuracy: 0.9979
Epoch 16/60
1632/1632 - 2s - loss: 0.0130 - accuracy: 0.9978
Epoch 17/60
1632/1632 - 2s - loss

In [39]:
#Quantify our trained Solved Model
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")


544/544 - 1s - loss: 0.0140 - accuracy: 0.9976
Normal Neural Network - Loss: 0.013983643613755703, Accuracy: 0.9975870251655579


In [40]:
#Make Prediction for the solved/unsolved murder
encoded_predictions = np.argmax(model.predict(X_test_scaled[:5]), axis = -1)
prediction_labels = sle.inverse_transform(encoded_predictions)

In [41]:
#Solved/Unsolved encoder reminder
print(solved_mappings)

{0: 'No', 1: 'Yes'}


In [42]:
print(f"Predicted Solved: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted Solved: ['Yes' 'Yes' 'Yes' 'Yes' 'Yes']
Actual Labels: [1, 1, 1, 1, 1]
